# Exploring your fisrt PIConGPU LWFA simulation

## load python modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as const
import openpmd_api as io
import openpmd_viewer as viewer

### add libaries shiped with PIConGPU 

Please adjust `PIConGPU_src` to the path to your PIConGPU source code

In [ ]:
PIConGPU_src = "__path_to_our_PIConGPU_source_code__"

In [ ]:
import sys
sys.path.append(PIConGPU_src + "/lib/python/picongpu/")

In [ ]:
# might need to be executed 2x !!! 

from extra.plugins.data import EnergyHistogramData

## set path to your simulation directory

Please adjust `path_to_simulation`

In [ ]:
path_to_simulation = "__path_to_your_PIConGPU_simulation__"

## load data from energy histogram (in-situ plugin)

In [ ]:
# create object for all energy histogram data
eh_data = EnergyHistogramData(path_to_simulation) # the directory in which simOutput is located

In [ ]:
# show available iterations
print("1:", eh_data.get_iterations(species='e'), "\n")

# show available simulation times
print("2.", eh_data.get_times(species='e'))

In [ ]:
# load data for a given iteration
counts, bins_keV, _, _ = eh_data.get(species='e', species_filter='all', iteration=2000)

### plot energy histogram 

In [ ]:
plt.plot(bins_keV, counts)

plt.xlabel(r"$E \mathrm{[MeV]}$", fontsize=18)
plt.xticks(fontsize=14)

plt.ylabel(r"$N_e$", fontsize=18)
plt.yticks(fontsize=14)
plt.yscale("log")

plt.tight_layout()
plt.show()

### some more options:

In [ ]:
# load data for a given time
counts, bins_keV, iteration, dt = eh_data.get(species='e', species_filter='all', time=1.3900e-14)

In [ ]:
# get data for multiple iterations
counts, bins_keV, iteration, dt = eh_data.get(species='e', iteration=[200, 400, 1000])

## load openPMD series

(no changes needed)

In [ ]:
series = io.Series(path_to_simulation + "/simOutput/openPMD/simData_%06T.bp", access=io.Access_Type.read_only)

### select a time step (iteration)

choose any valid output time step

In [ ]:
time_step = 1500

it = series.iterations[time_step]

### what attributes are there?

In [ ]:
for i in it.attributes:
    print(i)

let's get the grid resolution:

In [ ]:
delta_x = it.get_attribute('cell_width')
delta_y = it.get_attribute('cell_height')
unit_length = it.get_attribute('unit_length')

# convert to SI units
delta_x *= unit_length
delta_y *= unit_length

print(delta_x, delta_y) # in meter

### load mesh data

In [ ]:
# load electron density

h = it.meshes["e_all_chargeDensity"][io.Mesh_Record_Component.SCALAR]

print("shape:", h.shape)
N_x, N_y, N_z = h.shape

In [ ]:
# load a slice of the 3D data 

n_e = h[:, :, 192//2]
n_e_SI = h.unit_SI
series.flush()
n_e *= n_e_SI / const.elementary_charge * -1

### generate axis of mesh data

In [ ]:
x = (np.arange(N_x) - N_x//2) * delta_x
y = np.arange(N_y) * delta_y

### plot mesh data

In [ ]:
plt.pcolormesh(y/1e-6, x/1e-6, n_e/1e25, cmap=plt.cm.gray_r, vmax=3)

plt.xlabel(r"$y \, \mathrm{[\mu m]}$", fontsize=18)
plt.xticks(fontsize=14)

plt.ylabel(r"$x \, \mathrm{[\mu m]}$", fontsize=18)
plt.yticks(fontsize=14)

cb = plt.colorbar()
cb.set_label(r"$n_e \, \mathrm{[10^{25} m^{-3}]}$", fontsize=18)

plt.tight_layout()
plt.show()

### more meshes 

There are are moshes to explore:

In [ ]:
for i in it.meshes:
    print(i)

feel free to add your own code

## load particle data

In [ ]:
# macro particle weighting

h = it.particles["e"]["weighting"][io.Mesh_Record_Component.SCALAR]

print("number of particles:", h.shape)

w = h.load_chunk()
w_SI = h.unit_SI
series.flush()
w *= w_SI

In [ ]:
# momentum component

h = it.particles["e"]["momentum"]["y"]

p_y = h.load_chunk()
p_y_SI = h.unit_SI
series.flush()
p_y *= p_y_SI / w / const.electron_mass / const.speed_of_light

### plot particle data

In [ ]:
plt.hist(p_y, weights=w, bins=128)
plt.yscale("log")


plt.xlabel(r"$p_y /(m_e c)$", fontsize=18)
plt.xticks(fontsize=14)

plt.ylabel(r"$N_e$", fontsize=18)
plt.yticks(fontsize=14)

plt.tight_layout()
plt.show()

### there is more particle data to explore

In [ ]:
for i in it.particles["e"]:
    print(i)

feel free to writhe your own data analysis

In [ ]:
from extra.plugins.plot_mpl import EnergyHistogramMPL

In [ ]:
!ls /scratch/project_465001310/pauschri/08_LWFA_4_nores_mappedMemory/simOutput